In [15]:
import ee
import numpy as np

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-martinparker637')

In [22]:
#location = ee.Geometry.Point([10.05, 54.3167]) #(lon, lat)
coord = [54.3167, 10.05, 7504.0]
box_width = 0.002

location = ee.Geometry.BBox(coord[1] - box_width, coord[0] - box_width, coord[1] + box_width, coord[0] + box_width)
# Define the satellite imagery dataset (e.g., Landsat 8)
dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# Filter the dataset by location and date range
filtered_dataset = dataset.filterBounds(location).filterDate('2013-01-01', '2022-12-31')

In [3]:
# Define a function to calculate NDVI
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['B4', 'B8']).rename('NDVI')
    return image.addBands(ndvi)

# Map the NDVI calculation function over the filtered dataset
ndvi_dataset = filtered_dataset.map(calculate_ndvi)

In [7]:
ndvi_dataset

In [23]:
time_series_data = filtered_dataset.select('B4', 'B8').getRegion(location, 30).getInfo()

In [24]:
print(len(np.unique(np.array(time_series_data)[:, 3])), len(np.array(time_series_data)[:, 3]))

781 163801


In [6]:
# Get the time series data for the location
time_series_data = ndvi_dataset.getRegion(location, 30).getInfo()


EEException: Expected a homogeneous image collection, but an image with incompatible bands was encountered:
  First image type: 27 bands ([B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11, B12, AOT, WVP, SCL, TCI_R, TCI_G, TCI_B, MSK_CLDPRB, MSK_SNWPRB, QA10, QA20, QA60, MSK_CLASSI_OPAQUE, MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, NDVI]).
Current image type: 27 bands ([B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B11, B12, AOT, WVP, SCL, TCI_R, TCI_G, TCI_B, QA10, QA20, QA60, MSK_CLASSI_OPAQUE, MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, MSK_CLDPRB, MSK_SNWPRB, NDVI]).
          Image ID: 20170503T104021_20170503T104024_T32UNF
Some bands might require explicit casts.

In [ ]:

# Extract the time property separately and format it
time_values = [item[0] for item in time_series_data[1:]]
time_values = pd.to_datetime([item.split('_')[-1] for item in time_values], format='%Y%m%d')
# Create a pandas DataFrame from the time series data (excluding time)
df = pd.DataFrame([item[1:] for item in time_series_data[1:]], columns=time_series_data[0][1:])
df['time'] = time_values